# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""691197781971827891""","""../../../../data/SampleHDS.jso…",1.8131e19,"""6788814000666158344""","""1807495087070518929""","""10715696322800222803""",false
"""10999938515951544731""","""../../../../data/SampleHDS.jso…",4.4513e18,"""17766061654098553858""","""1807495087070518929""","""11790348815617421942""",true
"""17812041478982766094""","""../../../../data/SampleHDS.jso…",1.1866e19,"""543964326338920493""","""9707906583168779324""","""10715696322800222803""",false
"""13839515366937315848""","""../../../../data/SampleHDS.jso…",7.3997e17,"""17766061654098553858""","""12527926076214438504""","""10715696322800222803""",false
"""2628909978425021390""","""../../../../data/SampleHDS.jso…",1.3863e19,"""6788814000666158344""","""12527926076214438504""","""2272780766066799917""",true
"""2446656932817686084""","""../../../../data/SampleHDS.jso…",6.7232e18,"""543964326338920493""","""12317285103146212382""",null,false
"""16847043457161517924""","""../../../../data/SampleHDS.jso…",8.3425e18,"""543964326338920493""","""12527926076214438504""","""2272780766066799917""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""3263834468606255033""","""../../../../data/SampleHDS.jso…",7.4715e17,"""2286599241787763870""","""FirstMortgage30yr""","""16715406515172532053""","""Rejected"""
"""14874286154894550228""","""../../../../data/SampleHDS.jso…",6.1233e18,"""8666711214355202543""","""FirstMortgage30yr""","""4895876909422641951""","""Accepted"""
"""17177501560813984677""","""../../../../data/SampleHDS.jso…",7.6431e17,"""17588318256273102662""","""MoneyMarketSavingsAccount""","""16715406515172532053""","""Rejected"""
"""441329367583901785""","""../../../../data/SampleHDS.jso…",1.4396e19,"""8666711214355202543""","""BasicChecking""","""16715406515172532053""","""Rejected"""
"""5855532576063378990""","""../../../../data/SampleHDS.jso…",2.4408e18,"""2286599241787763870""","""BasicChecking""","""479697389844671758""","""Accepted"""
"""799813177046235496""","""../../../../data/SampleHDS.jso…",9.6075e18,"""17588318256273102662""","""UPlusFinPersonal""",null,"""Rejected"""
"""7600645980953877941""","""../../../../data/SampleHDS.jso…",1.7229e19,"""17588318256273102662""","""BasicChecking""","""479697389844671758""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""2510431148283100736""","""../../../../data/SampleHDS.jso…",1.5489e19,"""1168283601422785168""","""FirstMortgage30yr""","""3289888222901381477""","""Rejected"""
"""13544339998689241518""","""../../../../data/SampleHDS.jso…",1.5364e19,"""1466340910418875350""","""FirstMortgage30yr""","""5550077600777793491""","""Accepted"""
"""12198588379682162784""","""../../../../data/SampleHDS.jso…",2.9325e18,"""10191297706386385913""","""MoneyMarketSavingsAccount""","""3289888222901381477""","""Rejected"""
"""6480202717234652871""","""../../../../data/SampleHDS.jso…",1.1188e19,"""1466340910418875350""","""BasicChecking""","""3289888222901381477""","""Rejected"""
"""5629024877325767703""","""../../../../data/SampleHDS.jso…",2.7728e18,"""1168283601422785168""","""BasicChecking""","""6726269753998004695""","""Accepted"""
"""8975731957308671675""","""../../../../data/SampleHDS.jso…",1.2885e19,"""10191297706386385913""","""UPlusFinPersonal""",null,"""Rejected"""
"""17991739025603532050""","""../../../../data/SampleHDS.jso…",1.5425e19,"""10191297706386385913""","""BasicChecking""","""6726269753998004695""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""15122765976582220577""",8.6265e18,"""5762467642394820671""","""FirstMortgage30yr""","""4897140851006610""","""Rejected"""
"""13698617338135623329""",1.7737e19,"""9812330530823692772""","""FirstMortgage30yr""","""4894900476293963135""","""Accepted"""
"""12614204991113463875""",1.4366e19,"""9347960655015846029""","""MoneyMarketSavingsAccount""","""4897140851006610""","""Rejected"""
"""13601209821213943615""",8.4258e18,"""9812330530823692772""","""BasicChecking""","""4897140851006610""","""Rejected"""
"""4343123255841089993""",1.5715e19,"""5762467642394820671""","""BasicChecking""","""10226615252858444876""","""Accepted"""
"""5434870236914464649""",4.5615e18,"""9347960655015846029""","""UPlusFinPersonal""",null,"""Rejected"""
"""16471380837393918291""",1.7608e19,"""9347960655015846029""","""BasicChecking""","""10226615252858444876""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'